In [70]:
import ctypes
import os
import sys  
import numpy as np 
class pysperr:
    def __init__(self, libpath):
        self.sperr = ctypes.cdll.LoadLibrary(libpath)
        # self.sperr.sperr_comp_2d = self.sperr.sperr_comp_2d
        self.sperr.sperr_comp_2d.argtypes = (ctypes.c_void_p, ctypes.c_int32,
                                             ctypes.c_size_t, ctypes.c_size_t, ctypes.c_int32, ctypes.c_double,
                                             ctypes.POINTER(ctypes.c_void_p), ctypes.POINTER(ctypes.c_size_t))

        self.sperr.sperr_comp_2d.restype = ctypes.c_int32
        
        # self.sperr.sperr_comp_3d = self.sperr.sperr_comp_3d
        self.sperr.sperr_comp_3d.argtypes = [ctypes.c_void_p, ctypes.c_int32,
                            ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t, ctypes.c_int32, ctypes.c_double,
                            ctypes.POINTER(ctypes.c_void_p), ctypes.POINTER(ctypes.c_size_t)]
        self.sperr.sperr_comp_3d.restype = ctypes.c_int32
        
        # self.sperr.sperr_decomp_2d = self.sperr.sperr_decomp_2d
        self.sperr.sperr_decomp_2d.argtypes = [ctypes.c_void_p, ctypes.c_size_t, ctypes.c_int32,
                            ctypes.POINTER(ctypes.c_size_t), ctypes.POINTER(ctypes.c_size_t),
                            ctypes.POINTER(ctypes.c_void_p)]
        self.sperr.sperr_decomp_2d.restype = ctypes.c_int32
        
        # self.sperr.sperr_decomp_3d = self.sperr.sperr_decomp_3d
        self.sperr.sperr_decomp_3d.argtypes =[ctypes.c_void_p, ctypes.c_size_t, ctypes.c_int32,
                            ctypes.POINTER(ctypes.c_size_t), ctypes.POINTER(ctypes.c_size_t),ctypes.POINTER(ctypes.c_size_t),
                            ctypes.POINTER(ctypes.c_void_p)]
        self.sperr.sperr_decomp_3d.restype = ctypes.c_int32
        

    def compress(self, src, mode_str, quality):
        # src has to be a numpy array check at the beginning
        if not isinstance(src, np.ndarray):
            raise ValueError("Input has to be a numpy array")
        dst = ctypes.c_void_p()
        dst_len = ctypes.c_size_t()
        modes = {"bpp": 1, "psnr": 2, "pwe": 3}
        if mode_str not in modes:
            raise ValueError("Invalid mode")
        mode = modes[mode_str]
        is_float = 1 if src.dtype == np.float32 else 0
        shape = src.shape
        if len(src.shape) == 2:
            result = self.sperr.sperr_comp_2d(src.ctypes.data, is_float, shape[1], 
                           shape[0], mode, quality, ctypes.byref(dst), ctypes.byref(dst_len))
        elif len(src.shape) == 3:
            result = self.sperr.sperr_comp_3d(src.ctypes.data, is_float, shape[2],shape[1],shape[0], 
                         mode, quality, ctypes.byref(dst), ctypes.byref(dst_len))
        else:
            raise ValueError("Only 2D or 3D arrays are supported")
        if result != 0:
            raise ValueError("Compression failed")
        outbytes = ctypes.create_string_buffer(dst_len.value)
        ctypes.memmove(outbytes, dst, dst_len.value)
        ## free the memory of dst
        sperr_library.free(dst)
        dst = None
        dst_bytes = None
        # outbytes = np.frombuffer(outbytes, dtype=np.uint8)
        print(type(outbytes))
        return outbytes, dst_len.value
    
    def decompress(self,src, is_float, dims):
        src_ptr = ctypes.cast(src, ctypes.c_void_p)
        dtype = np.float32 if is_float == 1 else np.float64
        dst = ctypes.c_void_p()
        dtype = np.float32 if is_float == 1 else np.float64
        size = dims[0] * dims[1] * (dims[2] if len(dims) == 3 else 1)
        if len(dims) == 2:
            result = self.sperr.sperr_decomp_2d(src_ptr, ctypes.c_size_t(len(src)), 
                             ctypes.c_int32(is_float), 
                             ctypes.byref(ctypes.c_size_t(dims[1])),
                             ctypes.byref(ctypes.c_size_t(dims[0])), ctypes.byref(dst))
        elif len(dims) == 3:
            result = self.sperr.sperr_decomp_3d(src_ptr, ctypes.c_size_t(len(src)), 
                             ctypes.c_int32(is_float),  
                             ctypes.byref(ctypes.c_size_t(dims[2])),
                             ctypes.byref(ctypes.c_size_t(dims[1])),
                             ctypes.byref(ctypes.c_size_t(dims[0])), ctypes.byref(dst))
        else:
            raise ValueError("Only 2D or 3D arrays are supported")
        if result != 0:
            raise ValueError("Decompression failed")
        
        outbytes = ctypes.create_string_buffer(size *(4 if is_float else 8))
        ctypes.memmove(outbytes, dst, size *(4 if is_float else 8))
        dst_array = np.frombuffer(outbytes, dtype=dtype, count=size)
        sperr_library.free(dst)
        src_ptr = None
        dst = None
        outbytes = None
        return dst_array
    
    def compress_decompress(self, src, mode_str, quality):
        dst, dst_len = self.compress(src, mode_str, quality)
        is_float = 1 if src.dtype == np.float32 else 0
        dims = src.shape
        dst_array = self.decompress(dst, is_float, dims)
        result = {}
        result["ddata"] = dst_array.reshape(dims)
        result["cr"] = 1.0* src.nbytes/dst_len
        return result 
            
            

In [71]:
speerpy = pysperr('/Users/pjiao/git/sperr/build/src/libSPERR.dylib')
test_data = np.fromfile('/Users/pjiao/data/CESM_1800X3600/CLDHGH_1_1800_3600.dat', dtype=np.float32).reshape(1800, 3600)
compressed,det_len  = speerpy.compress(test_data, "pwe", 0.23)
print("cr =" , test_data.nbytes*1.0/det_len)

with open('dst.dat', 'wb') as f:
  f.write(compressed)
  
# print("cr =  ", (det_len*1.0/len(test_data) *4 ) )
decompressed = speerpy.decompress(compressed, 1, test_data.shape)

print("max = ", decompressed.max()) 
# decompressed.tofile('dat.sperr')

speerpy.compress_decompress(test_data, "pwe", 0.23)

<class 'ctypes.c_char_Array_3621'>
cr = 7158.2435791217895
max =  0.8974884
<class 'ctypes.c_char_Array_3621'>


{'ddata': array([[0.07282807, 0.07282003, 0.07280332, ..., 0.0897676 , 0.08976795,
         0.0897681 ],
        [0.07283264, 0.07282472, 0.07280825, ..., 0.08960362, 0.08960397,
         0.08960412],
        [0.07284141, 0.07283373, 0.07281778, ..., 0.08925814, 0.08925848,
         0.08925863],
        ...,
        [0.07832469, 0.07831854, 0.07830501, ..., 0.08479808, 0.0847991 ,
         0.08479953],
        [0.07828997, 0.07828382, 0.07827028, ..., 0.08476873, 0.08476975,
         0.08477019],
        [0.07827807, 0.07827192, 0.07825838, ..., 0.08475865, 0.08475967,
         0.08476011]], dtype=float32),
 'cr': 7158.2435791217895}

In [37]:
import ctypes
import numpy as np 

# Load the shared library (replace 'your_library.so' with the actual library file)
sperr_library = ctypes.CDLL('/Users/pjiao/git/sperr/build/src/libSPERR.dylib')
# int sperr_comp_2d(
#     const void* src,  /* Input: buffer that contains a 2D slice */
#     int32_t is_float, /* Input: input buffer type: 1 == float, 0 == double */
#     size_t dimx,      /* Input: X (fastest-varying) dimension */
#     size_t dimy,      /* Input: Y (slowest-varying) dimension */
#     int32_t mode,     /* Input: compression mode to use */
#     double quality,   /* Input: target quality */
#     void** dst,       /* Output: buffer for the output bitstream, allocated by this function */
#     size_t* dst_len);
# Define the argument and return types for the C functions

sperr_comp_2d = sperr_library.sperr_comp_2d
sperr_comp_2d.argtypes = [ctypes.c_void_p, ctypes.c_int32,
                            ctypes.c_size_t, ctypes.c_size_t, ctypes.c_int32, ctypes.c_double,
                            ctypes.POINTER(ctypes.c_void_p), ctypes.POINTER(ctypes.c_size_t)]



def compress_2d(src, is_float, dimx, dimy, mode, quality):
    # dst_bytes = ctypes.create_string_buffer(dimx * dimy *(4 if is_float else 8))
    # dst = ctypes.pointer(dst_bytes)
    dst = ctypes.c_void_p()
    dst_len = ctypes.c_size_t()
    ## get the content of the dst array 
    result = sperr_comp_2d(src.ctypes.data, is_float, dimx, 
                           dimy, mode, quality, ctypes.byref(dst), ctypes.byref(dst_len))
    if result != 0:
        raise ValueError("Error occurred during compression")

    # outbytes = dst_bytes.raw[:dst_len.value] # get the content of the dst array
    # free the memory of dst 
    outbytes = ctypes.create_string_buffer(dst_len.value)
    ctypes.memmove(outbytes, dst, dst_len.value)
    ## free the memory of dst
    sperr_library.free(dst)
    dst = None
    dst_bytes = None
    # outbytes = np.frombuffer(outbytes, dtype=np.uint8)
    print(type(outbytes))
    return outbytes, dst_len.value

# int sperr_decomp_2d(
#     const void* src,      /* Input: buffer that contains a compressed bitstream */
#     size_t src_len,       /* Input: length of the input bitstream in byte */
#     int32_t output_float, /* Input: output data type: 1 == float, 0 == double */
#     size_t* dimx,         /* Output: X (fast-varying) dimension */
#     size_t* dimy,         /* Output: Y (slowest-varying) dimension */
#     void** dst);          /* Output: buffer for the output 2D slice, allocated by this function */
speer_decomp_2d = sperr_library.sperr_decomp_2d 
speer_decomp_2d.argtypes = [ctypes.c_void_p, ctypes.c_size_t, ctypes.c_int32,
                            ctypes.POINTER(ctypes.c_size_t), ctypes.POINTER(ctypes.c_size_t),
                            ctypes.POINTER(ctypes.c_void_p)]
speer_decomp_2d.restype = ctypes.c_int32

def decompress_2d(src, is_float, dimx, dimy):
    ## src is created by ctypes.create_string_buffer()
    src_ptr = ctypes.cast(src, ctypes.c_void_p)
    dtype = np.float32 if is_float == 1 else np.float64
    # dst_array = np.zeros(dimx*dimy, dtype=dtype) 
    dst = ctypes.c_void_p()
    result = speer_decomp_2d(src_ptr, ctypes.c_size_t(len(src)), 
                             ctypes.c_int32(is_float),  
                             ctypes.byref(ctypes.c_size_t(dimx)), 
                             ctypes.byref(ctypes.c_size_t(dimy)), ctypes.byref(dst))
    if result != 0:
        raise ValueError("Error occurred during decompression")
    
    outbytes = ctypes.create_string_buffer(dimx * dimy *(4 if is_float else 8))
    ctypes.memmove(outbytes, dst, dimx * dimy *(4 if is_float else 8))
    dst_array = np.frombuffer(outbytes, dtype=dtype, count=dimx*dimy)
    sperr_library.free(dst)
    src_ptr = None
    dst = None
    outbytes = None
    return dst_array




In [38]:

test_data = np.fromfile('/Users/pjiao/data/CESM_1800X3600/CLDHGH_1_1800_3600.dat', dtype=np.float32)
compressed,det_len  = compress_2d(test_data, 1, 3600, 1800, 3, 1)
print(det_len)

with open('dst.dat', 'wb') as f:
  f.write(compressed)
  
# print("cr =  ", (det_len*1.0/len(test_data) *4 ) )
decompressed = decompress_2d(compressed, 1, 3600, 1800)

print("max = ", decompressed.max()) 
# decompressed.tofile('dat.sperr')


<class 'ctypes.c_char_Array_1092'>
1092
max =  0.87210906
